In [142]:
import os
import sys
import numpy as np
from random import randint

class HMM(object): 
    # base class for different HMM models
    def __init__(self, T, O, pi):
        # model is (T, O, pi) where T = Transition probs(hidden_states*hidden_states), 
        # O = Emission Probs(hidden_states*states), pi = initial distribution(hidden_states)               
        if T is None:
            print ("Error: You should provide the transition matrix")
            sys.exit() # Read in parameters from the file of model_name
        if O is None:
            print ("Error: You should provide the emission matrix")
            sys.exit() # Read in parameters from the file of model_name
        if pi is None:
            print ("Error: You should provide the initial probability")
            sys.exit() # Read in parameters from the file of model_name           
        self.pi=pi
        self.T=T
        self.O=O
        self.M=T.shape[1]  # M:number of hidden states of the model
        self.N=O.shape[1]  # N:number of states of the model

        
    def backward(self, obs):
        # This function is for backward algorithm, suppose that T, O, pi 
        #are given, and it calculates a bwk matrix (obs*states).
        # The backward algorithm can be used to calculate the likelihood 
        #of the probability P(Y_{k+1}, ... , Y_n|t_k=C)
        #=sum_q P(Y_{k+2}, ... , Y_n|t_{k+1}=q)P(q|C)P(x_{k+1}|q)
        #The backward probability b is the probability of seeing the observations from 
        #time t + 1 to the end, given that we are in state i at time t
        self.bwk = np.zeros(shape=(len(obs),self.M))
        # Initalize bwk to be empty matrix T*M
        # Initialize base cases (t == T)
        self.bwk[len(obs)-1,:]=np.ones(self.M)      
        for t in reversed(range(len(obs)-1)):
            self.bwk[t,:] = np.sum(self.bwk[t+1,:] * self.T[:,:] * self.O[:,obs[t+1]],axis=1)        
                #beta_k(C)=\sum_q beta_{k+1}(q)P(q|C)P(w_{k+1}|q)
        prob = np.sum(self.pi[0,:]* self.O[:,obs[0]] * self.bwk[0,:]) 
        return prob 
        #This prob is the likelihood of the input obs   
 

    def forward(self, obs):
        # This function is for forward algorithm, suppose that A, B, pi are given, 
        #and it calculates a fwd matrix (obs*states).
        # The forward algorithm can be used to calculate the likelihood of the model
        #P(Y1, ... , Yn)=sum_t(\prod_i P(Y[i]|t[i])P(t[i]|t[i-1])
        self.fwd = np.zeros(shape=(len(obs),self.M)) 
        #Initalize fwk to be empty matrix, and finally fwd is T*M
        # Initialize base cases (t == 0)
        self.fwd[0,:]=self.pi[0,:] * self.O[:,obs[0]] 
            #alpha_1(q)=p(w1,t1=q)=P(t1=q|t0)*p(w1|t1=q)
        # Run Forward algorithm for t > 0            
        for t in range(1, len(obs)):
            self.fwd[t,:] = np.sum(self.fwd[t-1,:] * self.T[:,:] * self.O[:,obs[t]],axis=1) 
                #alpha_k(q)=\sum_q1 alpha_{k-1}(q1)P(t_k=q|t_{k-1}=q1)P(w_k|t+k=q)
        prob = np.sum(self.fwd[len(obs) - 1,:]) 
        # The likelihood of input equals to the summation of fwd[N][t]
        return prob

    
    def viterbi(self, obs):
    #the task of determining which sequence of variables is the underlying source 
    #of some sequence of observations is called the decoding task
    #Decoding: Given as input an HMM = (A, B, pi) and a sequence of observations 
    #O = Y_1, ... Y_N, find the most probable sequence of states Q = X_1, ... X_T.
    # Goal: find the best path!
    # argmax_t P(Y1, ... Y_N, X_1, ..., X_T|A, B, pi)
        vit = np.zeros(shape=(len(obs),self.M))
        #[[0 for x in range(self.M)] for y in range(len(obs))] 
        # matrix
        path=np.zeros(shape=(len(obs),self.M))
        path[0,:]=range(self.M)
        # path
        # Initialize base cases (t == 0)
        vit[0,:] = self.pi[0,:] * self.O[:,obs[0]]    
        # Run Viterbi for t > 0      
        for t in range(1, len(obs)):
            vit[t,:]=np.max(vit[t-1,:] * self.T[:,:] * self.O[:,obs[t]],axis=1)
            path[t,:]=np.argmax(vit[t-1,:] * self.T[:,:] * self.O[:,obs[t]],axis=1)
        prob=np.max(vit[len(obs)-1,:])
        ind=np.argmax(vit[len(obs)-1,:])
        state=path[:,ind]
        #(prob, state) = max((vit[n][y], y) for y in range(self.M))
        return (prob, state)

    
    
    def forward_backward(self, obs): 
        #Output matrix gamma: gamma[t][y]=P(q_t=j|Y_1, ..., Y_N,A,B,pi)
        #and tensor zi: zi[t][i][j]=P(q_t=i,q_{t+1}=j|Y_1, ..., Y_N,A,B,pi)
        gamma = np.zeros(shape=(len(obs),self.M))
        # this is needed to keep track of finding a state i at a time t for all i and all t
        zi= np.zeros(shape=(len(obs),self.M,self.M))
        #zi = [[[0 for x in range(self.M)] for y in range(self.M)] for z in range(len(obs))]  
        # this is needed to keep track of finding a state i at a time t and j at a time (t+1) 
        #for all i and all j and all t
        # get alpha and beta tables computes
        p_obs = self.forward(obs)
        self.backward(obs)
        # compute gamma values
        for t in range(len(obs)):
            gamma[t,:] = (self.fwd[t,:] * self.bwk[t,:]) / p_obs
            if t == 0:
                self.pi[0,:] = gamma[t,:]
                #gamma[t][y]=P(q_t=j|Y_1, ..., Y_N,A,B,pi)
                #=P(q_t=j,Y_1, ..., Y_N|A,B,pi)/P(Y_1, ..., Y_N|A,B,pi)
                #=alpha_t(j)beta_t(j)/P(Y_1, ..., Y_N|A,B,pi)
                #compute zi values up to T - 1
            if t == len(obs) - 1:
                continue
            for y1 in range(self.M):
                zi[t,:,:] = self.fwd[t,:] * self.T[:,:] * self.O[:,obs[t + 1]] * self.bwk[t + 1,:] / p_obs
        #zi[t][i][j]=P(q_t=i,q_{t+1}=j|Y_1, ..., Y_N,A,B,pi)       
        #=P(q_t=i,q_{t+1}=j,Y_1, ..., Y_N|A,B,pi)/P(Y_1, ..., Y_N|A,B,pi)
        #=alpha_t(i)a_{ij}b_j(O_{t+1})beta_{t+1}(j)/apha_t(X_T)
        return (gamma,zi)
    
    
    def baum_welch(self,obs):
        #returns model given the initial model and observations  
        #The Baum-Welch algorithm iteratively estimate the counts.
        #We will start with an estimate for the transition and observation probabilities and 
        #then use these estimated probabilities to derive better and better probabilities. 
        #We get our estimated probabilities by computing the forward probability for 
        #an observation and then dividing that probability mass among all the different 
        #paths that contributed to this forward probability.
        gamma = np.zeros(shape=(len(obs),self.M))
        zi =  np.zeros(shape=(len(obs),self.M,self.M))
        # now that we have gamma and zi let us re-estimate
        (gamma,zi)=self.forward_backward(obs)
        
        #Update T
        #T_{ij)=expected number of transitions from state i to state j/expected number 
        #of transitions from state i
        a=np.sum(zi,axis=(0,2))
        self.T=np.sum(zi,axis=0)/np.array([a,]*self.M).transpose()
        
        for y in range(self.M):
            for k in range(self.N): 
                # for all symbols vk
                val = 0.0
                for t in range(len(obs)):
                    if obs[t] == k :
                        val += gamma[t][y]
                val /= np.sum(gamma[:,y])
                self.O[y][k] = val
                #O_j(v_k)=expected number of times in state j and observing symbol vk/expected 
                #number of times in state j
        return

In [143]:
M=3
N=3
T = np.array([[0, 0.2, 0.8],
[0, 0.6, 0.4],
[0, 0.5, 0.5]])
# Get transition probability

O = np.array([[0.33, 0.33, 0.33],
[0.2, 0.4, 0.4],
[0.5, 0.4, 0.1]])
# Get emission probability
pi_raw = np.random.random((1, M)) 
row_sums_pi = pi_raw.sum(axis=1)
pi = pi_raw / row_sums_pi[:, np.newaxis]

# Get initial probability
hmm=HMM(T,O,pi)

In [144]:
#hmm=HMM(M,N)
observations = []
observations = [0, 0, 1, 2, 2, 1]
print("M=", M, "N=", N, "Observations = ", observations)

M= 3 N= 3 Observations =  [0, 0, 1, 2, 2, 1]


In [145]:
p1=hmm.backward(observations)
print(" Bwk Prob = ", p1)

 Bwk Prob =  0.0014549839338601146


In [146]:
p2=hmm.forward(observations)
print(" Fwd Prob = ", p2)

 Fwd Prob =  0.0018602694040986056


In [147]:
prob, hidden_states = hmm.viterbi(observations)
print("Max Probability = ", prob, " Hidden State Sequence = ", hidden_states)

Max Probability =  0.0001489589889762435  Hidden State Sequence =  [0. 2. 2. 2. 1. 2.]
